# Problem Statement 


Data preparation for analysis will be done in this notebook

# Import packages

In [40]:
##for faster data processing
import numba as nb
import dask as da

#for data manipulation 
import pandas as pd
import numpy as np

from datetime import datetime

# Import dataset

In [41]:
#read the data file
query_df = pd.read_csv('../Sendy_data/query_result.csv', sep=',')
missed_cancellation_df = pd.read_csv('../Sendy_data/missing_cancellation_reason.csv', sep=',')


In [42]:
#Look at the last rows from the query_df dataset
query_df.tail(10)
#query_results_df.head()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
6396,Personal,NaN,Bike,Express,android,The driver is too far away.,Customer,7.0,15:50:35,7.0,15:57:13,7.0,15:58:21,10.0,-1.296175,36.825742,OnDemAND
6397,Personal,NaN,Bike,Standard,ios,No driver has been allocated to my request.,Customer,7.0,8:10:46,NaN,NaN,7.0,9:24:59,13.0,-1.233660,36.747580,OnDemAND
6398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
#Look at a couple of rows from the query_df dataset
query_df.sample(5)

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
2698,Personal,NaN,Bike,Express,android,The driver is too far away.,Customer,3.0,13:49:48,NaN,NaN,3.0,14:06:30,16.0,-1.187859,36.988853,OnDemAND
160,Business,39592.0,Bike,Express,desktop,I placed the wrong locations.,Customer,1.0,9:46:39,NaN,NaN,1.0,9:48:49,11.0,-1.284193,36.666223,OnDemAND
1888,Business,32692.0,Bike,Express,API,No Partner,Staff,2.0,15:40:27,NaN,NaN,3.0,3:00:15,0.0,-1.331390,36.865994,OnDemAND
2079,Business,20680.0,Bike,Standard,android,No driver has been allocated to my request.,Customer,3.0,15:31:25,3.0,15:46:14,3.0,16:15:43,16.0,-1.316254,36.702167,OnDemAND
2460,Business,14992.0,Bike,Express,desktop,My order is not ready.,Customer,3.0,15:39:21,3.0,15:39:41,3.0,15:40:04,8.0,-1.300820,36.764446,OnDemAND


In [44]:
query_df.head()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
0,Personal,NaN,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,17:55:38,NaN,NaN,1.0,17:56:35,9.0,-1.313053,36.841437,OnDemAND
1,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,12:26:27,1.0,12:27:36,1.0,12:28:44,14.0,-1.318959,36.853394,OnDemAND
2,Business,20680.0,Bike,Express,android,I placed the wrong locations.,Customer,1.0,13:21:18,1.0,13:21:34,1.0,13:47:29,16.0,-1.313854,36.706867,OnDemAND
3,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,10:29:19,NaN,NaN,1.0,10:29:29,20.0,-1.206197,36.778776,OnDemAND
4,Personal,NaN,Bike,Standard,ios,No Partner,Staff,1.0,14:34:05,1.0,14:59:08,1.0,15:49:41,11.0,-1.296958,36.808166,OnDemAND


In [45]:
# drop last n rows
#drop the created bottom rows with Nan
query_df.drop(query_df.tail(8).index,inplace=True) 


In [46]:
query_df.tail()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
6393,Personal,NaN,Bike,Express,android,No Partner,Staff,7.0,0:00:04,NaN,NaN,7.0,3:00:42,0.0,-1.301907,36.767142,OnDemAND
6394,Personal,NaN,Bike,Standard,android,No driver has been allocated to my request.,Customer,7.0,12:18:14,NaN,NaN,7.0,13:36:58,3.0,-1.256702,36.799419,OnDemAND
6395,Personal,NaN,Bike,Standard,android,The driver is too far away.,Customer,7.0,16:02:06,NaN,NaN,7.0,17:00:57,26.0,-1.267896,36.733265,OnDemAND
6396,Personal,NaN,Bike,Express,android,The driver is too far away.,Customer,7.0,15:50:35,7.0,15:57:13,7.0,15:58:21,10.0,-1.296175,36.825742,OnDemAND
6397,Personal,NaN,Bike,Standard,ios,No driver has been allocated to my request.,Customer,7.0,8:10:46,NaN,NaN,7.0,9:24:59,13.0,-1.233660,36.747580,OnDemAND


In [47]:
query_df.shape


(6398, 17)

In [48]:
missed_cancellation_df.drop(missed_cancellation_df.tail(8).index,inplace=True) 


In [49]:
missed_cancellation_df.tail()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
324,Personal,NaN,Bike,Express,android,NaN,Staff,7.0,0:00:04,NaN,NaN,7.0,3:00:48,0.0,-1.305507,36.775842,OnDemAND
325,Personal,NaN,Bike,Standard,android,NaN,Staff,7.0,6:15:12,7.0,8:27:31,7.0,10:49:54,3.0,-1.285342,36.770167,Scheduled
326,Personal,NaN,Bike,Standard,desktop,NaN,Customer,7.0,16:33:09,NaN,NaN,7.0,16:33:57,17.0,-1.274964,36.911953,OnDemAND
327,Personal,NaN,Bike,Standard,android,NaN,Customer,7.0,13:00:13,NaN,NaN,7.0,13:20:01,10.0,-1.324390,36.849949,OnDemAND
328,Personal,NaN,Bike,Standard,android,NaN,Customer,7.0,8:07:50,NaN,NaN,7.0,8:31:51,22.0,-1.297139,36.790044,OnDemAND


In [50]:
missed_cancellation_df.shape

(329, 17)

# Rename columns

**List columns**

In [51]:
query_df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Personal or Business', 'Business Number', 'Vehicle Type',
       'Standard or Express', 'Platform', 'Cancel Reason', 'Cancelled by',
       'Placement Day', 'Placement Time', 'Confirmation Day',
       'Confirmation Time', 'Cancellation Day', 'Cancellation Time',
       'Distance (KM)', 'Pickup Lat', 'Pickup Long', 'scheduled'],
      dtype='object')>

In [52]:
missed_cancellation_df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Personal or Business', 'Business Number', 'Vehicle Type',
       'Standard or Express', 'Platform', 'Cancel Reason', 'Cancelled by',
       'Placement Day', 'Placement Time', 'Confirmation Day',
       'Confirmation Time', 'Cancellation Day', 'Cancellation Time',
       'Distance (KM)', 'Pickup Lat', 'Pickup Long', 'scheduled'],
      dtype='object')>

In [53]:
# Rename columns using a dictionary to map values

query_df.rename(columns={'Personal or Business':'personal_or_Business', 'Business Number':'Business_Number', 'Vehicle Type':'Vehicle_Type', 
                                 'Standard or Express':'Standard_or_Express', 'Cancel Reason':'Cancel_Reason', 'Cancelled by':'Cancelled_by',
                                 'Placement Day':'Placement_Day', 'Placement Time':'placement_time', 'Confirmation Day':'Confirmation_Day',
                                'Confirmation Time':'Confirmation_Time', 'Cancellation Day':'Cancellation_Day', 'Cancellation Time':'Cancellation_time',
                                'Distance (KM)':'Distance_(KM)', 'Pickup Lat':'Pickup_Lat', 'Pickup Long':'Pickup_Long'}, inplace=True)

In [54]:
missed_cancellation_df.rename(columns={'Personal or Business':'personal_or_Business', 'Business Number':'Business_Number', 'Vehicle Type':'Vehicle_Type', 
                                 'Standard or Express':'Standard_or_Express', 'Cancel Reason':'Cancel_Reason', 'Cancelled by':'Cancelled_by',
                                 'Placement Day':'Placement_Day', 'Placement Time':'placement_time', 'Confirmation Day':'Confirmation_Day',
                                'Confirmation Time':'Confirmation_Time', 'Cancellation Day':'Cancellation_Day', 'Cancellation Time':'Cancellation_time',
                                'Distance (KM)':'Distance_(KM)', 'Pickup Lat':'Pickup_Lat', 'Pickup Long':'Pickup_Long'}, inplace=True)

In [55]:
missed_cancellation_df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['personal_or_Business', 'Business_Number', 'Vehicle_Type',
       'Standard_or_Express', 'Platform', 'Cancel_Reason', 'Cancelled_by',
       'Placement_Day', 'placement_time', 'Confirmation_Day',
       'Confirmation_Time', 'Cancellation_Day', 'Cancellation_time',
       'Distance_(KM)', 'Pickup_Lat', 'Pickup_Long', 'scheduled'],
      dtype='object')>

In [56]:
#change all the column names to lower case names
query_df.columns = map(str.lower, query_df.columns)

In [57]:
query_df.columns

Index(['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled'],
      dtype='object')

In [58]:
#change all the column names to lower case names
missed_cancellation_df.columns = map(str.lower, missed_cancellation_df.columns)

In [59]:
missed_cancellation_df.columns

Index(['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled'],
      dtype='object')

# Check for Nan

In [60]:
query_df.isnull().any()

personal_or_business    False
business_number          True
vehicle_type            False
standard_or_express      True
platform                False
cancel_reason           False
cancelled_by            False
placement_day           False
placement_time          False
confirmation_day         True
confirmation_time        True
cancellation_day        False
cancellation_time       False
distance_(km)           False
pickup_lat              False
pickup_long             False
scheduled               False
dtype: bool

Null values are in 3 variables, the business number, confirmation_day and confirmation_time

In [61]:
query_df.isnull().sum()

personal_or_business       0
business_number         3334
vehicle_type               0
standard_or_express        5
platform                   0
cancel_reason              0
cancelled_by               0
placement_day              0
placement_time             0
confirmation_day        3516
confirmation_time       3516
cancellation_day           0
cancellation_time          0
distance_(km)              0
pickup_lat                 0
pickup_long                0
scheduled                  0
dtype: int64

In [62]:
missed_cancellation_df.isnull().any()

personal_or_business    False
business_number          True
vehicle_type            False
standard_or_express     False
platform                False
cancel_reason            True
cancelled_by            False
placement_day           False
placement_time          False
confirmation_day         True
confirmation_time        True
cancellation_day        False
cancellation_time       False
distance_(km)           False
pickup_lat              False
pickup_long             False
scheduled               False
dtype: bool

In [63]:
missed_cancellation_df.isnull().sum()

personal_or_business      0
business_number         169
vehicle_type              0
standard_or_express       0
platform                  0
cancel_reason           329
cancelled_by              0
placement_day             0
placement_time            0
confirmation_day        223
confirmation_time       223
cancellation_day          0
cancellation_time         0
distance_(km)             0
pickup_lat                0
pickup_long               0
scheduled                 0
dtype: int64

According to observation both data sets have null values on the same columns i.e. business_number, confirmation_day,confirmation_time
except for the missed cancellation datasets which has null values on the cancel reason column. this will be worked on later

**Observe if the nature of the null values and if the nulls fall under the same indexes**

In [64]:
query_df[query_df.isnull().values.any(axis=1)]

,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,confirmation_day,confirmation_time,cancellation_day,cancellation_time,distance_(km),pickup_lat,pickup_long,scheduled
0,Personal,NaN,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,17:55:38,NaN,NaN,1.0,17:56:35,9.0,-1.313053,36.841437,OnDemAND
1,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,12:26:27,1.0,12:27:36,1.0,12:28:44,14.0,-1.318959,36.853394,OnDemAND
3,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,10:29:19,NaN,NaN,1.0,10:29:29,20.0,-1.206197,36.778776,OnDemAND
4,Personal,NaN,Bike,Standard,ios,No Partner,Staff,1.0,14:34:05,1.0,14:59:08,1.0,15:49:41,11.0,-1.296958,36.808166,OnDemAND
5,Personal,NaN,Bike,Express,ios,Customer error,Staff,1.0,7:37:19,1.0,7:38:09,1.0,8:03:28,7.0,-1.291352,36.772931,OnDemAND
6,Personal,NaN,Bike,Standard,android,No driver has been allocated to my request.,Customer,1.0,10:11:45,NaN,NaN,1.0,10:15:04,16.0,-1.212616,36.908138,OnDemAND
7,Personal,NaN,Bike,Standard,desktop,No driver has been allocated to my request.,Customer,1.0,16:10:52,NaN,NaN,2.0,7:44:42,21.0,-1.257130,36.699467,Scheduled
8,Personal,NaN,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,18:09:06,1.0,18:11:37,1.0,18:11:49,10.0,-1.278756,36.821739,OnDemAND
9,Business,20680.0,Bike,Express,desktop,No driver has been allocated to my request.,Customer,1.0,16:01:11,NaN,NaN,1.0,16:48:49,11.0,-1.223622,36.808150,OnDemAND
12,Business,52840.0,Bike,Express,desktop,I placed the wrong locations.,Customer,1.0,14:52:03,NaN,NaN,1.0,14:52:18,2.0,-1.274105,36.802374,OnDemAND


In [65]:
missed_cancellation_df[missed_cancellation_df.isnull().values.any(axis=1)]

,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,confirmation_day,confirmation_time,cancellation_day,cancellation_time,distance_(km),pickup_lat,pickup_long,scheduled
0,Personal,NaN,Bike,Express,ios,NaN,Customer,1.0,15:23:44,1.0,16:40:47,1.0,16:43:50,4.0,-1.299370,36.822107,OnDemAND
1,Business,20680.0,Bike,Express,desktop,NaN,Staff,1.0,14:21:46,1.0,14:22:20,1.0,14:30:48,17.0,-1.264490,36.789303,OnDemAND
2,Business,17968.0,Bike,Express,desktop,NaN,Customer,1.0,15:13:20,NaN,NaN,1.0,15:56:34,18.0,-1.427056,36.961499,OnDemAND
3,Personal,NaN,Bike,Standard,android,NaN,Customer,1.0,13:54:19,NaN,NaN,1.0,13:56:41,8.0,-1.301737,36.743936,OnDemAND
4,Personal,NaN,Bike,Standard,android,NaN,Staff,1.0,13:24:05,1.0,15:57:20,1.0,17:14:46,13.0,-1.308098,36.920464,Scheduled
5,Personal,NaN,Bike,Express,android,NaN,Customer,1.0,17:25:37,NaN,NaN,1.0,19:03:28,6.0,-1.290903,36.795503,OnDemAND
6,Personal,NaN,Bike,Standard,android,NaN,Customer,1.0,10:00:42,1.0,10:42:28,1.0,11:31:24,6.0,-1.268865,36.765106,OnDemAND
7,Business,44464.0,Bike,Express,desktop,NaN,Customer,1.0,11:50:46,1.0,11:51:15,1.0,11:51:17,60.0,-1.325435,36.869036,OnDemAND
8,Personal,NaN,Bike,Standard,ios,NaN,Customer,1.0,16:00:06,NaN,NaN,1.0,16:00:46,14.0,-1.215739,36.835509,Scheduled
9,Personal,NaN,Bike,Express,ios,NaN,Customer,1.0,10:35:26,1.0,11:41:36,1.0,11:43:58,5.0,-1.253696,36.795834,OnDemAND


## Deal with the null values 

**Business number**

In [66]:
#replace all the missing values with -1
query_df['business_number'].fillna(-1, inplace=True)

**confirmation_day**

In [67]:
query_df['confirmation_day'].fillna(-1, inplace=True)

In [68]:
#with_business_number=query_df[query_df['business_number'].notnull()].reset_index()
#without_business_number=query_df[query_df['business_number'].isnull()].reset_index()

In [69]:
missed_cancellation_df['business_number'].fillna(-1, inplace=True)

In [70]:
missed_cancellation_df['confirmation_day'].fillna(-1, inplace=True)

In [71]:
query_df['standard_or_express'].fillna('missing', inplace=True)

In [72]:
query_df.isnull().any().values

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False])

In [73]:
query_df['standard_or_express'].value_counts()

Express     4013
Standard    2380
missing        5
Name: standard_or_express, dtype: int64

In [74]:
query_df=query_df[query_df['standard_or_express']!='missing']

# Change data types

**A data type, in programming, is a classification that specifies which type of value 
a variable has and what type of mathematical, relational or logical operations can be applied to it without causing an error.** 


1. The Personal or Business is a object but it should be an bool
2. The business_number is a float64 but it should be an int64
3. The Standard or Express is a float64 but it should be an bool.
4. The placement_time, confirmation_time, cancellation_time is a object instead of datetime64[ns]
5. **The cancelled_by is a object but it should be an bool
6. **The vehicle_type is a object but it should be an bool
7. **The scheduled is a object but it should be an bool
8. **The placement_day is a float but it should be an int64
9. **The confirmation_day is a float but it should be an int64
10. **The cancellation_day is a float but it should be an int64
11. The distance_(km) is a float64 but it should be an int64
convert data types in pandas, Use astype() to force an appropriate dtype

In [75]:
query_df['cancelled_by'].value_counts()

Customer    4692
Staff       1701
Name: cancelled_by, dtype: int64

In [76]:
query_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6393 entries, 0 to 6397
Data columns (total 17 columns):
personal_or_business    6393 non-null object
business_number         6393 non-null float64
vehicle_type            6393 non-null object
standard_or_express     6393 non-null object
platform                6393 non-null object
cancel_reason           6393 non-null object
cancelled_by            6393 non-null object
placement_day           6393 non-null float64
placement_time          6393 non-null object
confirmation_day        6393 non-null float64
confirmation_time       2879 non-null object
cancellation_day        6393 non-null float64
cancellation_time       6393 non-null object
distance_(km)           6393 non-null float64
pickup_lat              6393 non-null float64
pickup_long             6393 non-null float64
scheduled               6393 non-null object
dtypes: float64(7), object(10)
memory usage: 899.0+ KB


In [77]:
query_df.get_dtype_counts()

float64     7
object     10
dtype: int64

The total number of entries is 6398 with both categorical and numerical data. 

In [78]:
query_df['business_number']=query_df['business_number'].astype('int64', inplace=True)

### datetime

**query_df**

In [79]:
# Convert date to pandas date_time
query_df['placement_time'] =  query_df['placement_time'].apply(pd.to_datetime, errors='coerce') 
query_df['confirmation_time'] =  query_df['confirmation_time'].apply(pd.to_datetime, errors='coerce') 
query_df['cancellation_time'] =  query_df['cancellation_time'].apply(pd.to_datetime, errors='coerce') 


In [80]:
#show total number of values of the column cancellation day
query_df['confirmation_day'].value_counts()

-1.0    3514
 4.0     586
 5.0     555
 2.0     522
 3.0     497
 1.0     423
 6.0     227
 7.0      69
Name: confirmation_day, dtype: int64

### days

In [81]:
query_df['cancellation_day'] =  query_df['cancellation_day'].astype('int64', inplace=True) 
query_df['placement_day'] =  query_df['cancellation_day'].astype('int64', inplace=True) 
query_df['confirmation_day'] =  query_df['cancellation_day'].astype('int64', inplace=True) 


#### Add columns for weekday and weekend names

In [95]:
query_df['business_number']=query_df['business_number'].astype('object', inplace=True)

**missed_cancellation_df**

In [98]:
missed_cancellation_df['business_number']=missed_cancellation_df['business_number'].astype('object', inplace=True)

In [99]:
# Convert date to pandas date_time
missed_cancellation_df['placement_time'] =  missed_cancellation_df['placement_time'].apply(pd.to_datetime, errors='coerce') 
missed_cancellation_df['confirmation_time'] =  missed_cancellation_df['confirmation_time'].apply(pd.to_datetime, errors='coerce') 
missed_cancellation_df['cancellation_time'] =  missed_cancellation_df['cancellation_time'].apply(pd.to_datetime, errors='coerce') 


**Change the days dtype float to int64**

In [100]:
missed_cancellation_df['cancellation_day'] =  missed_cancellation_df['cancellation_day'].astype('int64', inplace=True) 
missed_cancellation_df['placement_day'] =  missed_cancellation_df['cancellation_day'].astype('int64', inplace=True) 
missed_cancellation_df['confirmation_day'] =  missed_cancellation_df['cancellation_day'].astype('int64', inplace=True) 


### Distance 

**Change the distance_(km) dtype float to int64**

In [101]:
query_df['distance_(km)'] =  query_df['distance_(km)'].astype('int64', inplace=True) 


In [102]:
missed_cancellation_df['distance_(km)'] =  missed_cancellation_df['distance_(km)'].astype('int64', inplace=True) 


In [103]:
query_df['cancel_reason']=query_df['cancel_reason'].astype('object', inplace=True)

In [104]:
#show the dimensions
query_df.ndim

2

In [105]:
#show the final number of data types
query_df.get_dtype_counts()

datetime64[ns]    3
float64           2
int64             4
object            8
dtype: int64

In [106]:
query_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6393 entries, 0 to 6397
Data columns (total 17 columns):
personal_or_business    6393 non-null object
business_number         6393 non-null object
vehicle_type            6393 non-null object
standard_or_express     6393 non-null object
platform                6393 non-null object
cancel_reason           6393 non-null object
cancelled_by            6393 non-null object
placement_day           6393 non-null int64
placement_time          6393 non-null datetime64[ns]
confirmation_day        6393 non-null int64
confirmation_time       2879 non-null datetime64[ns]
cancellation_day        6393 non-null int64
cancellation_time       6393 non-null datetime64[ns]
distance_(km)           6393 non-null int64
pickup_lat              6393 non-null float64
pickup_long             6393 non-null float64
scheduled               6393 non-null object
dtypes: datetime64[ns](3), float64(2), int64(4), object(8)
memory usage: 899.0+ KB


In [92]:
missed_cancellation_df.get_dtype_counts()

datetime64[ns]    3
float64           3
int64             5
object            6
dtype: int64

# Export CSV

In [ ]:
query_df.to_csv("../Sendy_data/analysis_df.csv", sep="|")

In [ ]:
missed_cancellation_df.to_csv("../Sendy_data/training_df.csv", sep="|")